In [2]:
from kubeflow import fairing
from kubeflow.fairing.kubernetes import utils as k8s_utils

PRIVATE_REGISTRY = 'registry.kube-system.svc.cluster.local:30000'

fairing.config.set_preprocessor(
    'python', 
     # command = ['python'],  # default: python
     input_files = ['00-python-file-to-fairing.py'], # 생성되는 image의 /app/ 디렉토리로 복사됨
)

fairing.config.set_builder(
    'append',
    base_image = 'tensorflow/tensorflow',
    registry = PRIVATE_REGISTRY,
    image_name='my-02-python-file-fairing', 
    push=True
)

fairing.config.set_deployer(
    'job',
    # namespace='myspace', # default: 현재 네임스페이스
    pod_spec_mutators=[
        k8s_utils.get_resource_mutator(cpu=1, memory=5)]
)

fairing.config.run()

[I 210314 18:49:29 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.base.BasePreProcessor object at 0x7f03006fe470>
[I 210314 18:49:29 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f03006fecf8>
[I 210314 18:49:29 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f03006fee80>
[W 210314 18:49:29 append:50] Building image using Append builder...
[I 210314 18:49:29 base:107] Creating docker context: /tmp/fairing_context_n6ovkg5v
[I 210314 18:49:29 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow'
[W 210314 18:49:30 append:54] Image successfully built in 0.49828900400098064s.
[W 210314 18:49:30 append:94] Pushing image registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:63233498...
[I 210314 18:49:30 docker_creds_:234] Loading Docker credentials for repository 'registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:63233498'
[W 

2021-03-14 18:49:31.692006: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-03-14 18:49:31.692055: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
11493376/11490434 [==============================] - 0s 0us/step
2021-03-14 18:49:34.677229: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-03-14 18:49:34.677505: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-03-14 18:49:34.677532: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-03-14 18:49:34.677558: I tensorflow/stream_executor/cuda/cuda

[W 210314 18:50:07 job:173] Cleaning up job fairing-job-jvj8f...


(<kubeflow.fairing.preprocessors.base.BasePreProcessor at 0x7f03006fe470>,
 <kubeflow.fairing.deployers.job.job.Job at 0x7f03006fee80>)

# curl로 프라이빗 레지스트리 확인

In [3]:
! curl http://registry.kube-system.svc.cluster.local:30000/v2/_catalog

{"repositories":["my-02-python-file-fairing","my-03-function-fairing-job","mybase","mymnistbase","mytfjob","tfjob-fairing-mnist"]}


In [5]:
! curl http://registry.kube-system.svc.cluster.local:30000/v2/my-02-python-file-fairing/tags/list

{"name":"my-02-python-file-fairing","tags":["729DED65","20F8BE01","2EEA161F","388750DC","63233498","16CF3292"]}


# Image 확인
- Host에서 실행
- fairing에서 난수로 생성한 tag를 변경해서 실행할 것 (예: E2683407)

### Image pull
```sh
docker pull registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```

### Image inspect
```sh
docker inspect registry.kube-system.svc.cluster.local:30000/my-02-python-file-fairing:E2683407
```
Cmd 항목을 확인하고 faring 코드대로 image build 되었음을 확인
```json
...
            "Cmd": [
                "python",
                "/app/00-python-file-to-fairing.py"
            ],
...
```